In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Examine Train/Test Sets

In [2]:
df_train_sets = pd.read_pickle('ebw_poc_train_sample_expt3_with_shf_aligned_h1.pkl')
df_test_sets  = pd.read_pickle('ebw_poc_test_sample_expt3_with_shf_aligned_h1.pkl')

In [55]:
pd.concat([df_train_sets.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].describe(), df_test_sets.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].describe()], axis=1)

count        mean         std  min    25%  \
weeks_since_hit censoring_flg_mod                                              
0.0             False              305.0  914.255738  912.276516  1.0  244.0   
                True               717.0  909.008368  573.162303  8.0  430.0   
1.0             False              305.0  866.613115  890.652703  1.0  208.0   
                True               717.0  839.200837  560.242646  6.0  367.0   
2.0             False              305.0  816.406557  869.391097  1.0  171.0   
...                                  ...         ...         ...  ...    ...   
155.0           False                5.0    1.000000    0.000000  1.0    1.0   
                True                 1.0    7.000000         NaN  7.0    7.0   
156.0           False                2.0    1.000000    0.000000  1.0    1.0   
157.0           False                2.0    1.000000    0.000000  1.0    1.0   
158.0           False                2.0    1.000000    0.000000  1.0    1.0   

                                     50%     75%     max  count        mean  \
weeks_since_hit censoring_flg_mod                                             
0.0             False              565.0  1262.0  3688.0  103.0  885.893204   
                True               867.0  1298.0  2481.0  239.0  967.175732   
1.0             False              504.0  1209.0  3595.0  103.0  840.281553   
                True               779.0  1212.0  2394.0  239.0  898.845188   
2.0             False              448.0  1121.0  3497.0  103.0  793.592233   
...                                  ...     ...     ...    ...         ...   
155.0           False                1.0     1.0     1.0    2.0    1.000000   
                True                 7.0     7.0     7.0    NaN         NaN   
156.0           False                1.0     1.0     1.0    2.0    1.000000   
157.0           False                1.0     1.0     1.0    2.0    1.000000   
158.0           False                1.0     1.0     1.0    2.0    1.000000   

                                          std   min    25%    50%     75%  \
weeks_since_hit censoring_flg_mod                                           
0.0             False              858.584208   4.0  232.5  566.0  1344.0   
                True               633.310923  14.0  433.0  856.0  1442.5   
1.0             False              841.171678   4.0  200.0  514.0  1284.5   
                True               619.497090  10.0  394.0  798.0  1379.0   
2.0             False              824.320997   2.0  170.5  483.0  1211.0   
...                                       ...   ...    ...    ...     ...   
155.0           False                0.000000   1.0    1.0    1.0     1.0   
                True                      NaN   NaN    NaN    NaN     NaN   
156.0           False                0.000000   1.0    1.0    1.0     1.0   
157.0           False                0.000000   1.0    1.0    1.0     1.0   
158.0           False                0.000000   1.0    1.0    1.0     1.0   

                                      max  
weeks_since_hit censoring_flg_mod          
0.0             False              3210.0  
                True               2462.0  
1.0             False              3169.0  
                True               2405.0  
2.0             False              3136.0  
...                                   ...  
155.0           False                 1.0  
                True                  NaN  
156.0           False                 1.0  
157.0           False                 1.0  
158.0           False                 1.0  

[315 rows x 16 columns]

In [3]:
df_cuml_spins=pd.read_pickle('ebw_raw_features_base_cuml_spins_h1.pkl')

In [4]:
df_cuml_spins['cuml_song_station_spins_non_on_since_hit'] = df_cuml_spins['cuml_song_station_spins_non_on'] - df_cuml_spins['cuml_song_station_spins_non_on_at_hit']

In [5]:
mask = (df_cuml_spins['weeks_since_hit'] >= 0) & (df_cuml_spins['weeks_since_hit'] <= 26)
df_sub = df_cuml_spins.loc[mask]

In [10]:
df_sub.groupby(['weeks_since_hit']).apply(
    lambda x: pd.qcut(x['cuml_song_station_spins_non_on_at_hit'], q=5, labels=False)).droplevel(0)

19        3.0
47        0.0
94        1.0
137       1.0
176       0.0
         ... 
653670    3.0
653758    3.0
653976    3.0
654105    3.0
654474    1.0
Name: cuml_song_station_spins_non_on_at_hit, Length: 87493, dtype: float64

In [6]:
df_sub['at_hit_quintile'] = df_sub.groupby(['weeks_since_hit']).apply(
    lambda x: pd.qcut(x['cuml_song_station_spins_non_on_at_hit'], q=5, labels=False)).droplevel(0)
# df_sub['since_hit_quintile'] = df_sub.groupby(['weeks_since_hit']).apply(
#     lambda x: pd.qcut(x['cuml_song_station_spins_non_on_since_hit'], q=5, labels=False)).droplevel(0)

/tmp/ipykernel_4247/1754467671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['at_hit_quintile'] = df_sub.groupby(['weeks_since_hit']).apply(


In [7]:
df_train_sets = df_train_sets.join(df_sub.set_index(['mediabase_id', 'call_letters', 'week_dt', 'weeks_since_hit'])['at_hit_quintile'], how='left')

In [8]:
df_test_sets = df_test_sets.join(df_sub.set_index(['mediabase_id', 'call_letters', 'week_dt', 'weeks_since_hit'])['at_hit_quintile'], how='left')

In [14]:
df_train_sets

,,,,song_format_spins_am_pm_dr_26wk_lag1_std,artist_market_spins_non_on_4wk_lag1_min,completed_rate_13wk,song_station_share_26wk_lag2,song_market_spins_non_on_delta_26wk_lag1_max,artist_format_ddl_1wk_lag1_mean,song_station_spins_non_on_pct_delta_26wk_lag2_max,artist_format_spins_am_pm_dr_8wk_lag1_min,song_market_spins_non_on_delta_4wk_lag2_median,song_station_spins_am_pm_dr_8wk_lag2_mean,...,thumbs_down_count_13wk_lag2_max,thumbs_up_count_4wk_lag1_min,song_format_spins_non_on_1wk_lag1_mean,artist_station_spins_non_on_4wk_lag1_mean,song_station_ddl_delta_13wk_lag2_max,song_station_ddl_pct_delta_4wk_lag2_max,completed_rate_8wk,censoring_flg_mod,target,at_hit_quintile
mediabase_id,call_letters,week_dt,weeks_since_hit,,,,,,,,,,,,,,,,,,,,,
2720314,WZFT-FM,2022-04-17,0.0,4085.365153,273.0,NaN,0.023007,20.0,13.0,3.000000,21770.0,0.5,51.500000,...,0.0,0.0,15263.0,234.50,18.0,1.800000,NaN,True,1905.0,3.0
2722189,KBKS-FM,2021-10-10,0.0,3214.443637,79.0,NaN,0.015350,74.0,15.0,5.000000,1467.0,33.0,24.125000,...,0.0,0.0,12980.0,64.50,6.0,0.800000,NaN,True,485.0,1.0
2693277,KZZP-FM,2021-05-30,0.0,1796.030922,40.0,NaN,0.011718,33.0,8.0,1.500000,514.0,16.5,8.142857,...,0.0,0.0,8332.0,38.25,11.0,0.000000,NaN,True,1408.0,0.0
2546146,WHTZ-FM,2020-11-08,0.0,3486.337605,18.0,0.623427,0.013154,49.0,18.0,2.000000,3057.0,1.0,0.875000,...,2712.0,9720.0,3703.0,1.75,8.0,0.571429,0.612895,True,209.0,3.0
2647678,WDCG-FM,2021-02-21,0.0,3216.618054,113.0,NaN,0.027064,42.0,15.0,5.500000,5162.0,6.5,48.375000,...,0.0,0.0,11836.0,93.50,3.0,0.187500,NaN,False,535.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337260,WRVW-FM,2022-01-09,26.0,32.911723,16.0,0.612335,0.003092,16.0,17.0,2.333333,934.0,-0.5,1.125000,...,3668.0,13412.0,710.0,2.25,5.0,0.454545,0.587120,False,157.0,4.0
2687270,KHTS-FM,2021-11-14,26.0,1568.837971,161.0,NaN,0.034171,50.0,13.0,1.055556,578.0,-4.5,30.750000,...,0.0,0.0,825.0,39.00,2.0,0.000000,NaN,False,1888.0,1.0
2580292,KZHT-FM,2021-11-07,26.0,523.153463,155.0,NaN,0.005912,28.0,15.0,2.428571,1338.0,-1.5,0.000000,...,56.0,48.0,1196.0,0.00,9.0,0.818182,NaN,True,4.0,4.0


In [9]:
pd.concat([df_train_sets.groupby(['weeks_since_hit'])['target'].count(), df_test_sets.groupby(['weeks_since_hit'])['target'].count()], axis=1)

,target,target
weeks_since_hit,,
0.0,1023,341
1.0,1023,341
2.0,1011,337
3.0,1005,336
4.0,978,326
5.0,963,321
6.0,928,310
7.0,903,302
8.0,870,291


In [5]:
df_train_sets.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].count()/df_train_sets.groupby(['weeks_since_hit'])['target'].count()

weeks_since_hit  censoring_flg_mod
0.0              False                0.296369
                 True                 0.703631
1.0              False                0.297059
                 True                 0.702941
2.0              False                0.300595
                 True                 0.699405
3.0              False                0.302395
                 True                 0.697605
4.0              False                0.310769
                 True                 0.689231
5.0              False                0.314583
                 True                 0.685417
6.0              False                0.326486
                 True                 0.673514
7.0              False                0.333333
                 True                 0.666667
8.0              False                0.346021
                 True                 0.653979
9.0              False                0.353011
                 True                 0.646989
10.0             False   

In [ ]:
df_extract_train = df_train_sets[df_train_sets.index.get_level_values(3) == 5]
df_extract_test =  df_test_sets[df_test_sets.index.get_level_values(3) == 5]

In [ ]:
df_extract_train.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].count()

In [ ]:
df_extract_test.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].count()

In [ ]:
df_extract_train['target'].plot(kind='hist')

In [ ]:
df_extract_train['target'].apply(lambda x: np.log(x)).plot(kind='hist')

## Model Training

In [3]:
# import sksurv and sklearn packages
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, RandomSurvivalForest
from sksurv.metrics import as_concordance_index_ipcw_scorer, as_cumulative_dynamic_auc_scorer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, auc
from sklearn.model_selection import RandomizedSearchCV, StratifiedGroupKFold, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
import time
import pickle
from feature_engine.selection import DropCorrelatedFeatures

#### define prereqs

In [4]:
id_cols = list(df_train_sets.index)
target_cols = ['censoring_flg_mod', 'target']
cat_cols =['market_name']
# cat_cols = ['market_name', 'at_hit_quintile']
feature_cols = list(set(df_train_sets.columns) - set(target_cols))
num_cols = list(set(feature_cols) -set(cat_cols))

In [5]:
split_number = 5
random_state = 1000
weeks_range = range(27)

# hyperparam grid for ElasticNet Baseline
param_grid_cnet = {
    'estimator__l1_ratio': [0.5, 0.1, 0.01, 0.001]
}

# hyperparam grid for Gradient Boosted Model
param_grid_gbm = {
    'estimator__max_depth': [2, 4, 6, 8, 10],
    'estimator__max_features': ['auto', 'sqrt', 'log2', None],
    'estimator__n_estimators': [50, 100, 150, 200],
    'estimator__learning_rate': [0.01, 0.001, 0.5, 0.1],
    'estimator__loss': ['coxph'],
    'estimator__subsample': [0.70, 0.80, 0.9, 1.0],
    'estimator__min_samples_split': [2, 4, 8, 16],
    'estimator__min_samples_leaf': [2, 4, 8, 16]
}

# hyperparam grid for Random Survival Forest
param_grid_rsf = {
    'estimator__max_depth': [2, 4, 6, 8, 10],
    'estimator__max_features': ['auto', 'sqrt', 'log2', None],
    'estimator__n_estimators': [50, 100, 150, 200],
    'estimator__min_samples_split': [2, 4, 8, 16, 32],
    'estimator__min_samples_leaf': [2, 4, 8, 16, 32],
    'estimator__max_leaf_nodes': [2, 4, 8, 16, 32],
    'estimator__bootstrap': [True, False],
    'estimator__oob_score': [False, True],
    'estimator__max_samples': [0.70, 0.80, 0.90, 1.0]
}

# hyperparam for GBM classifier
param_grid_gbm_clf = {
    'max_depth': [2, 4, 6, 8, 10],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.001, 0.5, 0.1],
    # 'loss': ['log_loss', 'exponential'],
    'subsample': [0.70, 0.80, 0.9, 1.0],
    'min_samples_split': [2, 4, 8, 16],
    'min_samples_leaf': [2, 4, 8, 16]
}

# best scores - train/test
best_scores_cnet_train = {}
best_scores_gbm_train = {}
best_scores_rsf_train = {}

best_scores_cnet_test = {}
best_scores_gbm_test = {}
best_scores_rsf_test = {}

# best estimators
best_estimators_cnet = {}
best_estimators_gbm = {}
best_estimators_rsf = {}

# pickle file names
file_cnet='best_est_cnet_h1_expt3_with_aligned_shf_nocorr.pkl'
file_gbm='best_est_gbm_h1_expt3_with_aligned_shf_nocorr.pkl'
file_rsf='best_est_rsf_h1_expt3_with_aligned_shf_nocorr.pkl'

#### train a baseline ElasticNet model

In [7]:
for wk in weeks_range:
    start = time.perf_counter()

    sv = CoxnetSurvivalAnalysis()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    drop_cols = X_train.columns[X_train.isna().any()].tolist()
    X_train.drop(drop_cols, axis=1, inplace=True)
    X_train = pd.get_dummies(X_train, columns=['market_name'])
    sel_cols = list(set(num_cols) - set(drop_cols))
    X_train[sel_cols] = StandardScaler().fit_transform(X_train[sel_cols])

    dcf= DropCorrelatedFeatures(threshold=0.7)
    X_train = dcf.fit_transform(X_train)

    y_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols].to_records(index=False)

    cv = KFold(n_splits=split_number, shuffle=True)
    sv_model = RandomizedSearchCV(estimator=as_concordance_index_ipcw_scorer(sv), param_distributions=param_grid_cnet,
                                  cv=cv, verbose=1, n_iter=4, n_jobs=-1)
    sv_model.fit(X_train, y_train)
    best_estimators_cnet[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))


    X_test = df_test_sets[df_test_sets.index.get_level_values(3) == wk][feature_cols]
    X_test = pd.get_dummies(X_test, columns=['market_name'])
    missing_cols = list(set(sv_model.best_estimator_.estimator.feature_names_in_) - set(X_test.columns))
    print(missing_cols)
    for col in missing_cols:
        X_test[col] = 0
    test_idx = X_test[sv_model.best_estimator_.estimator.feature_names_in_].dropna().index
    y_test = df_test_sets[df_test_sets.index.get_level_values(3) == wk].loc[test_idx, target_cols].to_records(index=False)

    print(str(wk) + '_' + 'test_score: ' + f'{sv_model.best_estimator_.score(X_test[sv_model.best_estimator_.estimator.feature_names_in_].loc[test_idx], y_test)}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
0_best_score: 0.7207829510680445
Elapsed time: 4
[]
0_test_score: 0.4659885694887854
Fitting 5 folds for each of 4 candidates, totalling 20 fits
1_best_score: 0.716413438968135
Elapsed time: 4
[]
1_test_score: 0.4640577737888606
Fitting 5 folds for each of 4 candidates, totalling 20 fits
2_best_score: 0.7141160311614859
Elapsed time: 3
[]
2_test_score: 0.4936175111406107
Fitting 5 folds for each of 4 candidates, totalling 20 fits
3_best_score: 0.7123862283819228
Elapsed time: 3
[]
3_test_score: 0.5111917957882369
Fitting 5 folds for each of 4 candidates, totalling 20 fits
4_best_score: 0.694939040107635
Elapsed time: 3
[]
4_test_score: 0.5028163540801616
Fitting 5 folds for each of 4 candidates, totalling 20 fits
5_best_score: 0.7065386338756643
Elapsed time: 3
[]
5_test_score: 0.49830343588506537
Fitting 5 folds for each of 4 candidates, totalling 20 fits
6_best_score: 0.699030843638532
Elapsed time: 3
[]
6_test_score: 0.5039

In [89]:
sv_model.best_params_

{'estimator__l1_ratio': 0.5}

In [84]:
y_test

rec.array([(False, 1515.), (False, 1491.), (False,  673.), (False,  208.),
           (False,  101.), (False, 1635.), (False,  358.), (False, 1461.),
           (False,  269.), (False, 2572.), (False, 1378.), (False,  413.),
           (False,  600.), (False, 3071.), (False,  267.), (False,   91.),
           (False, 1149.), (False, 2891.), (False,  868.), (False, 1770.),
           (False,  506.), (False,  335.), (False,  425.), (False,  790.),
           (False,  424.), (False, 2704.), (False,  323.), (False, 1310.),
           (False,  319.), (False,  529.), (False,  990.), (False,  860.),
           (False,  258.), (False, 1010.), (False,  342.), (False, 1630.),
           (False,  679.), (False,  847.), (False,  299.), (False,  232.),
           (False,  791.), (False,  233.), (False, 2569.), (False, 2959.),
           (False,  172.), (False,  220.), (False,   61.), (False,  118.),
           (False, 1599.), (False, 1020.), (False, 2551.), (False,   59.),
           (False, 2950.)

In [8]:
X_train.columns[X_train.isna().any()].tolist()

['completed_rate_26wk',
 'thumbs_up_rate_total_13wk',
 'thumbs_up_rate_1wk',
 'completed_rate_13wk',
 'thumbs_up_rate_total_26wk',
 'thumbs_up_rate_26wk',
 'completed_rate_4wk',
 'thumbs_up_rate_4wk',
 'thumbs_up_rate_total_1wk',
 'thumbs_up_rate_8wk',
 'thumbs_up_rate_total_4wk',
 'thumbs_up_rate_total_8wk',
 'completed_rate_8wk',
 'thumbs_up_rate_13wk',
 'completed_rate_1wk']

In [38]:
# write best estimators to a pickle file
with open(file_cnet, 'wb') as file:
    pickle.dump(best_estimators_cnet, file)

### train a GBM model

In [8]:
# train a gbm
times = np.arange(20,1010, 10)
for wk in weeks_range:
    start = time.perf_counter()

    sv = GradientBoostingSurvivalAnalysis()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    drop_cols = X_train.columns[X_train.isna().any()].tolist()
    X_train.drop(drop_cols, axis=1, inplace=True)
    X_train = pd.get_dummies(X_train, columns=['market_name'])


    dcf= DropCorrelatedFeatures(threshold=0.7)
    X_train = dcf.fit_transform(X_train)

    df_y_temp = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols]
    y_train = df_y_temp.to_records(index=False)

    # train gbm model
    cv = KFold(n_splits=split_number, shuffle=True)
    sv_model = RandomizedSearchCV(estimator=as_concordance_index_ipcw_scorer(sv, 1000), param_distributions=param_grid_gbm,
                                  cv=cv, verbose=1, n_iter=50, n_jobs=-1)

    # sv_model = RandomizedSearchCV(estimator=as_cumulative_dynamic_auc_scorer(sv, times), param_distributions=param_grid_gbm,
    #                               cv=split_number, verbose=1, n_iter=50, n_jobs=-1)

    sv_model.fit(X_train, y_train)
    best_estimators_gbm[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))

    X_test = df_test_sets[df_test_sets.index.get_level_values(3) == wk][feature_cols]
    X_test = pd.get_dummies(X_test, columns=['market_name'])
    missing_cols = list(set(sv_model.best_estimator_.estimator.feature_names_in_) - set(X_test.columns))
    # print(missing_cols)
    for col in missing_cols:
        X_test[col] = 0
    test_idx = X_test[sv_model.best_estimator_.estimator.feature_names_in_].dropna().index
    y_test = df_test_sets[df_test_sets.index.get_level_values(3) == wk].loc[test_idx, target_cols].to_records(index=False)

    print(str(wk) + '_' + 'test_score: ' + f'{sv_model.best_estimator_.score(X_test[sv_model.best_estimator_.estimator.feature_names_in_].loc[test_idx], y_test)}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
0_best_score: 0.7678980573899497
Elapsed time: 40
0_test_score: 0.7962497473657871
Fitting 5 folds for each of 50 candidates, totalling 250 fits
1_best_score: 0.7613367015173241
Elapsed time: 42
1_test_score: 0.7630600813917788
Fitting 5 folds for each of 50 candidates, totalling 250 fits
2_best_score: 0.7474127000380252
Elapsed time: 43
2_test_score: 0.7613612866641993
Fitting 5 folds for each of 50 candidates, totalling 250 fits
3_best_score: 0.7547744160000283
Elapsed time: 51
3_test_score: 0.7847659199034301
Fitting 5 folds for each of 50 candidates, totalling 250 fits
4_best_score: 0.7455173706552571
Elapsed time: 51
4_test_score: 0.7764421259311449
Fitting 5 folds for each of 50 candidates, totalling 250 fits
5_best_score: 0.7545537943751384
Elapsed time: 40
5_test_score: 0.7718480616562635
Fitting 5 folds for each of 50 candidates, totalling 250 fits
6_best_score: 0.7441706491082055
Elapsed time: 44
6_test_score: 0.76

In [9]:
# write best estimators to a pickle file
with open(file_gbm, 'wb') as file:
    pickle.dump(best_estimators_gbm, file)

#### train a gmb classifier

In [34]:
# train a gbm classifier
for wk in weeks_range:
    start = time.perf_counter()

    sv = GradientBoostingClassifier()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    drop_cols = X_train.columns[X_train.isna().any()].tolist()
    X_train.drop(drop_cols, axis=1, inplace=True)
    X_train = pd.get_dummies(X_train, columns=['market_name'])

    dcf= DropCorrelatedFeatures(threshold=0.7)
    X_train = dcf.fit_transform(X_train)

    df_y_temp = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols[0]]
    y_train = df_y_temp
    # y_train = df_y_temp.to_records(index=False)

    # train gbm model
    sv_model = RandomizedSearchCV(estimator=sv, param_distributions=param_grid_gbm_clf, scoring='roc_auc',
                                  cv=split_number, verbose=1, n_iter=10, n_jobs=-1)
    sv_model.fit(X_train, y_train)
    best_estimators_gbm[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0_best_score: 0.7429377051303281
Elapsed time: 35
Fitting 5 folds for each of 10 candidates, totalling 50 fits
1_best_score: 0.7603969868928886
Elapsed time: 21
Fitting 5 folds for each of 10 candidates, totalling 50 fits
2_best_score: 0.7769146998411038
Elapsed time: 73
Fitting 5 folds for each of 10 candidates, totalling 50 fits
3_best_score: 0.7892861046057769
Elapsed time: 68
Fitting 5 folds for each of 10 candidates, totalling 50 fits
4_best_score: 0.7987278362286926
Elapsed time: 43
Fitting 5 folds for each of 10 candidates, totalling 50 fits
5_best_score: 0.8012965343587295
Elapsed time: 25
Fitting 5 folds for each of 10 candidates, totalling 50 fits
6_best_score: 0.8072415864621894
Elapsed time: 35
Fitting 5 folds for each of 10 candidates, totalling 50 fits
7_best_score: 0.8179676684881603
Elapsed time: 75
Fitting 5 folds for each of 10 candidates, totalling 50 fits
8_best_score: 0.8207856559285928
Elapsed time: 32
F

In [33]:
X_train

,,,,artist_format_spins_am_pm_dr_13wk_lag2_std,thumbs_down_count_13wk_lag1_max,thumbs_up_count_26wk_lag2_max,song_format_spins_non_on_13wk_lag1_mean,song_market_spins_non_on_4wk_lag2_mean,market_spins_ddl_sens_26wk_lag1_median,completed_count_8wk_lag1_min,song_format_spins_am_pm_dr_1wk_lag2_mean,artist_format_spins_am_pm_dr_26wk_lag2_std,thumbs_up_count_8wk_lag2_mean,...,market_name_Pittsburgh,"market_name_Portland, OR",market_name_Raleigh,market_name_Salt Lake City,market_name_San Diego,market_name_San Francisco,market_name_Seattle,market_name_St. Louis,market_name_Tampa,"market_name_Washington, DC"
mediabase_id,call_letters,week_dt,weeks_since_hit,,,,,,,,,,,,,,,,,,,,,
2638062,WXXL-FM,2021-01-17,1.0,1596.437681,0.0,0.0,11359.769231,164.25,0.526984,6.0,7588.0,2240.410293,0.000,...,0,0,0,0,0,0,0,0,0,0
2691407,WHTZ-FM,2022-01-02,1.0,1156.636115,0.0,0.0,7063.230769,67.75,-0.670139,6.0,6559.0,1657.672376,0.000,...,0,0,0,0,0,0,0,0,0,0
2831030,WKST-FM,2022-11-20,1.0,1948.095645,0.0,0.0,6056.615385,170.50,0.000000,6.0,7030.0,2015.214139,0.000,...,1,0,0,0,0,0,0,0,0,0
2797282,WXKS-FM,2022-05-22,1.0,1962.706974,0.0,0.0,7105.800000,103.00,-0.000000,6.0,5915.0,1962.706974,0.000,...,0,0,0,0,0,0,0,0,0,0
2550239,KBKS-FM,2020-08-16,1.0,1100.998876,972.0,7137.0,7634.769231,195.50,0.058642,232641.0,7640.0,1229.682976,6472.125,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2697903,WWPW-FM,2021-08-22,1.0,3734.987674,0.0,0.0,13773.153846,179.75,0.198611,6.0,10583.0,4859.487387,0.000,...,0,0,0,0,0,0,0,0,0,0
2550239,WIOQ-FM,2020-06-07,1.0,381.599367,1088.0,5067.0,3326.538462,80.75,1.376290,204808.0,3649.0,1099.133665,4706.000,...,0,0,0,0,0,0,0,0,0,0
2720462,WDCG-FM,2021-09-19,1.0,2561.793684,0.0,0.0,12386.076923,162.00,0.419872,6.0,10756.0,2561.793684,0.000,...,0,0,1,0,0,0,0,0,0,0


#### train a RandomSurvivalForest

In [7]:
# train a gbm
for wk in weeks_range:
    start = time.perf_counter()

    sv = RandomSurvivalForest()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    drop_cols = X_train.columns[X_train.isna().any()].tolist()
    X_train.drop(drop_cols, axis=1, inplace=True)
    X_train = pd.get_dummies(X_train, columns=['market_name'])


    dcf= DropCorrelatedFeatures(threshold=0.7)
    X_train = dcf.fit_transform(X_train)

    df_y_temp = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols]
    y_train = df_y_temp.to_records(index=False)

    # train rsf model
    cv = KFold(n_splits=split_number, shuffle=True)
    sv_model = RandomizedSearchCV(estimator=as_concordance_index_ipcw_scorer(sv), param_distributions=param_grid_rsf,
                                  cv=cv, verbose=1, n_iter=50, n_jobs=-1)
    sv_model.fit(X_train, y_train)
    best_estimators_rsf[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))

    X_test = df_test_sets[df_test_sets.index.get_level_values(3) == wk][feature_cols]
    X_test = pd.get_dummies(X_test, columns=['market_name'])
    missing_cols = list(set(sv_model.best_estimator_.estimator.feature_names_in_) - set(X_test.columns))
    # print(missing_cols)
    for col in missing_cols:
        X_test[col] = 0
    test_idx = X_test[sv_model.best_estimator_.estimator.feature_names_in_].dropna().index
    y_test = df_test_sets[df_test_sets.index.get_level_values(3) == wk].loc[test_idx, target_cols].to_records(index=False)

    print(str(wk) + '_' + 'test_score: ' + f'{sv_model.best_estimator_.score(X_test[sv_model.best_estimator_.estimator.feature_names_in_].loc[test_idx], y_test)}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/ubuntu/PycharmProjects/adds-early-burnout-warning-model/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ubuntu/PycharmProjects/adds-early-burnout-warning-model/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ubuntu/PycharmProjects/adds-early-burnout-warning-model/venv/lib/python3.8/site-packages/sksurv/metrics.py", line 766, in fit
    self.estimator_ = self.estimator.fit(X, y, **fit_params)
  Fi

0_best_score: 0.7095324075023023
Elapsed time: 2460
0_test_score: 0.7130665297608472
Fitting 5 folds for each of 50 candidates, totalling 250 fits


KeyboardInterrupt: 

In [ ]:
# write best estimators to a pickle file
with open(file_rsf, 'wb') as file:
    pickle.dump(best_estimators_rsf, file)